In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

# prepare data
---

In [ ]:
ls -la ../input

In [ ]:
dir_data_src = '../input/anime-recommendations-database'
os.listdir(dir_data_src)

## load anime.csv

In [ ]:
anime_csv = pd.read_csv(os.path.join(dir_data_src, 'anime.csv'))
print(anime_csv.shape)
anime_csv.index = anime_csv.anime_id.values
anime_csv.head()

In [ ]:
np.sort(anime_csv.anime_id.values)

In [ ]:
# anime_csv_dic = {}
# for ii in range(anime_csv.shape[0]):
#     anime_id = anime_csv.anime_id.values[ii]
#     anime_name = anime_csv.name.values[ii]
#     anime_csv_dic[anime_id] = anime_name

# anime_csv_dic

## load rating.csv

In [ ]:
'''
load rating.csv
'''
rating_csv = pd.read_csv(os.path.join(dir_data_src, 'rating.csv'))
print(rating_csv.shape)
rating_csv.head()

In [ ]:
'''
`Yuri!!! on Ice` doesn't exist
'''
rating_csv.query('anime_id=="32995"')

In [ ]:
np.unique(rating_csv.user_id.values), len(np.unique(rating_csv.user_id.values))

In [ ]:
np.unique(rating_csv.rating.values)

In [ ]:
rating_csv.rating.values != -1

## create rating_csv2

In [ ]:
rating_csv2 = rating_csv.loc[rating_csv.rating.values != -1]
rating_csv2.shape
rating_csv2.head(10)

## Anime no-rated

In [ ]:
rating_csv2.groupby('anime_id').size().keys().values

In [ ]:
anime_csv.anime_id.values

In [ ]:
np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)

In [ ]:
'''
anime_id no rated
'''
anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]

In [ ]:
'''
how much
'''
anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)].shape

## show anime no-rated

In [ ]:
'''
show anime no rated
'''
anime_id_norated = anime_csv.anime_id.values[~np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]
print(anime_csv.loc[anime_id_norated].shape)
anime_csv.loc[anime_id_norated]

In [ ]:
rating_csv2.query('anime_id in @anime_id_norated')

## create anime_csv2
delete anime no-rated

In [ ]:
tmp = anime_csv.anime_id.values[np.isin(anime_csv.anime_id.values, rating_csv2.groupby('anime_id').size().keys().values)]
anime_csv2 = anime_csv.loc[tmp].copy()
print(anime_csv2.shape)
anime_csv2

# dic_anime
* use anime_csv2

In [ ]:
#['anime_id-'+str(ee) for ee in anime_csv.anime_id.values]
#dic_anime = gensim.corpora.Dictionary([['anime_id-'+str(ee) for ee in anime_csv2.anime_id.values]], prune_at=None)
dic_anime = gensim.corpora.Dictionary([['PL_ANIME']], prune_at=None)
dic_anime.add_documents([['anime_id-'+str(ee) for ee in anime_csv2.anime_id.values]])
print(len(dic_anime))
dic_anime

In [ ]:
dic_anime.doc2bow(['anime_id-1', 'anime_id-1', 'anime_id-5', 'anime_id-6'])

In [ ]:
dic_anime[0], dic_anime.doc2idx(['anime_id-1', 'anime_id-6'])

# user_dic

In [ ]:
user_dic = {}
for ee in np.unique(rating_csv.user_id.values):
    user_dic[ee] = 'user_id-' + str(ee)

len(user_dic)

In [ ]:
np.unique(rating_csv2.user_id.values)

In [ ]:
dic_user = gensim.corpora.Dictionary([['PL_USER']], prune_at=None)
dic_user.add_documents([['user_id-' + str(ee) for ee in np.unique(rating_csv2.user_id.values)]])
#dic_user.add_documents([['genre-'+ee for ee in genre]])
len(user_dic), len(dic_user)

In [ ]:
dic_user[0], dic_user[1], dic_user[2]

# create tfidf of user side

In [ ]:
from collections.abc import Sequence

class UserSeq(Sequence):
    '''
    row : anime
    col : user
    '''
    def __init__(self, anime_csv=anime_csv2, rating_csv=rating_csv2, dic_anime=dic_anime, dic_user=dic_user):
        self.anime_csv = anime_csv
        self.rating_csv = rating_csv
        self.dic_anime = dic_anime # doc
        self.dic_user = dic_user # word
        
        print(self.anime_csv.shape, len(self.dic_anime))
        assert (1+self.anime_csv.shape[0]) == len(self.dic_anime)
    
    def __getitem__(self, idx):
        return self._get(idx)
    
    del_user_id = re.compile('^user_id-')
    def _get(self, idx):
        try:
            user_id = self.del_user_id.sub('', self.dic_user[idx])
        except KeyError:
            raise IndexError
        
        if idx == 0:
            return [self.dic_anime[0]]
        animes = []
        for irow in self.rating_csv.query('user_id==@user_id').itertuples():
            anime_id = 'anime_id-' + str(irow.anime_id)
            rating = irow.rating
            animes.extend([anime_id]*rating)
        
        return animes
    
    def __len__(self):
        return len(self.dic_user)

In [ ]:
dic_anime[0], dic_anime[1]

In [ ]:
anime_csv.query("name=='Kimi no Na wa.'")

In [ ]:
anime_csv.query("name=='Kimi no Na wa.'").anime_id.values

In [ ]:
dic_anime.token2id['anime_id-32281']

In [ ]:
user_seq = UserSeq()
len(user_seq)

In [ ]:
user_seq[0]

In [ ]:
user_seq[dic_user.token2id['user_id-10001']][:20]

In [ ]:
len(user_seq[dic_user.token2id['user_id-10001']])

In [ ]:
%%time
corpus_csr = gensim.matutils.corpus2csc(
    (dic_anime.doc2bow(ee) for ee in user_seq), num_terms=max(dic_anime.keys())+1
).T
print('corpus_csr.shape >>>', corpus_csr.shape)

In [ ]:
%%time
tfidf = gensim.models.TfidfModel((dic_anime.doc2bow(ee) for ee in user_seq), id2word=dic_anime)
tfidf

In [ ]:
scipy.sparse.save_npz('corpus_csr', corpus_csr)
tfidf.save('tfidf')
dic_user.save('dic_user')
dic_anime.save('dic_anime')

In [ ]:
os.listdir('.')